# Notebook 04: Reporte Final - Política de Inventario 2024
## Objetivo: Combinar el pronóstico base (GBR) con el cálculo de riesgo (GPD por producto) para generar la política de inventario final.

## Metodología:

- Cargar el pronóstico base de 2024 (data/processed/predicciones_demanda_2024.csv).

- Cargar el mapeo de stock de seguridad por producto (data/processed/stock_seguridad_por_producto.csv).

- Unir (merge) las dos tablas.

- Calcular la columna final: stock_total_recomendado = prediccion_ventas + stock_de_seguridad.

- Guardar el reporte final.